In [ ]:
# Install dependencies
!pip install pandas matplotlib numpy scikit-learn

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib notebook

import pandas as pd                      # for handling tabular data
from sklearn import (cluster,            # for clustering algorithms
                     decomposition,      # for reducing dimensionality of data
                     manifold)           # for reducing dimensionality of data
import matplotlib.pyplot as plt          # plotting library
from mpl_toolkits.mplot3d import Axes3D  # 3D plots

from utils import (animate_dataframes,   # generates animated plots
                   aggregate_over_time)  # generates means over timestamp across days

## Getting data

Smart building systems have an array of sensors that periodically generate measurements. Each sensor is connected to a network and is addressable. The communication protocol to sensors is called [BACNet][3]. Vanderbilt has a dashboard that aggregates various BACNet points into surveys which can be plotted or viewed as tables.

This dashboard is called the [BuildingLogix Data eXchange portal (BDX) and is hosted at vanderbilt][1]. However the portal is only available from the vanderbit network. You can set up a [virtual private network access to vanderbilt through VUIT][2]. This is optional, in case you need to look at the kind of data we can download.

On BDX, a survey has been created which records electric meter readings for Engineering Science Building.

Links:

1. Data eXchange portal: https://facilities.app.vanderbilt.edu/trendview/
2. Link to actual survey: https://facilities.app.vanderbilt.edu/trendview/?open=2721

[1]: https://facilities.app.vanderbilt.edu/trendview/
[2]: https://it.vanderbilt.edu/services/catalog/end-point_computing/network_access/remote-access/index.php
[3]: https://en.wikipedia.org/wiki/BACnet

In [ ]:
# Load dataset
filepath = './ESB_Electric.csv'
df = pd.read_csv(filepath, index_col='time', parse_dates=True)
# Downsampling interval for faster processing. Default interval is '5min'
# Use offsets at which to downsample for e.g. '15min', '1H'
# See more offsets here: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases
df = df.asfreq('15min')

nan_rows = df.isnull().any(1)
len_total = len(df)
len_nan = nan_rows.sum()
# Remove rows with NaN values. The dataset contains some cells where values are missing.
# You can try to impute them or ignore them altogether. Which is better?
df.dropna(inplace=True)
len_valid = len(df)
# Sort by column names
df.sort_index(axis='columns', inplace=True)
print('{:10d} Total rows'.format(len_total))
print('{:10d} Rows with at least one NaN value'.format(len_nan))
print('{:10d} Rows with no NaN values'.format(len_valid))
df.head()

## Description of dataset

The dataset is provided in a CSV file ('ESB_Electric.csv'). It contains electric meter measurements of power demand at various points in the Engineering Science Building. Readings are collected at 5 minute intervals. The dataset contains measuremtns for January through December of 2019.

* `Humidity` is in percent (0-100).
* `Temperature` is in Degrees Faranheit
* `PowerDemand*` fields are in kW
* `Time` is in UTC. Measurements range from `2019-01-01 06:00Z` to `2020-01-01 06:00Z` (UTC). Or, in local (CST) time: `2019-01-01 00:00-6` to `2020-01-01 00:00-6`.

Each field in the dataset corresponds to a BACNet point. The addresses are:

| Field         	| Path                                                                              	|
|:---------------	|:-----------------------------------------------------------------------------------	|
| PowerDemand   	| /Drivers/NiagaraNetwork/VUZone5/points/ESB/Meters/ElectricMeterProfile            	|
| PowerDemand_* 	| /Drivers/NiagaraNetwork/VUZone5/points/ESB/SubMeters/ESB_*/ElectricMeterProfile   	|
| Temperature   	| /Weather Data/Nashville_Weather                                                 	|
| Humidity      	| /Weather Data/Nashville_Weather                                                 	|

You can use these BACNet addresses to enquire more about the dataset.

In [ ]:
# Show the distribution of fields in the dataset:
df.describe()

In [ ]:
power_cols = [col for col in df.columns if col.startswith('Power')]
weather_cols = ['Temperature', 'Humidity']

In [ ]:
# Grouping the data by week and taking means over each time
weekly_means, week_labels = aggregate_over_time(df, period='W')

## Trend plots

In [ ]:
# Environmental conditions
ax = plt.subplot(111)
anim = animate_dataframes(frames=weekly_means, labels=week_labels, ax=ax,
                          lseries=('Temperature',), rseries=('Humidity',),
                          ylabel=('Temperature /F', 'Humidity /%'),
                          ylim=((0, 110), (0, 100)),
                          xlabel='Time of day',
                          anim_args={'repeat': False})

In [ ]:
# Power measurements
ax = plt.subplot(111)
anim = animate_dataframes(frames=weekly_means, labels=week_labels, ax=ax,
                          lseries=power_cols,
                          xlabel='Time of day', ylabel='Power /kW',
                          ylim=(0, 1000),
                          anim_args={'repeat': False})

## Relationships

### Clustering

For information on clustering algorithms, see [Scikit-Learn documentation][1].

For information on principal component analysis (PCA) , see [Scikit-learn docomentation][2].

[1]: https://scikit-learn.org/stable/modules/clustering.html
[2]: https://scikit-learn.org/stable/modules/decomposition.html#pca

In [ ]:
# Weather data
weather = df[weather_cols]
clusterer = cluster.DBSCAN()
cluster_labels = clusterer.fit_predict(weather.values)

plt.scatter(x=weather['Temperature'],
            y=weather['Humidity'],
            c=cluster_labels,
            marker=',', s=0.1)
plt.xlabel('Temperature /F')
plt.ylabel('Humidity /%')

In [ ]:
# Power data
power = df[power_cols]
# The data is high dimensional (> 2 fields) so cannot be plotted
# on a 2D plot. So first, we project it to 2 dimensions using
# principal component analysis.
mapper = decomposition.PCA(n_components=3)
lowdim = mapper.fit_transform(power.values)
clusterer = cluster.DBSCAN()
cluster_labels = clusterer.fit_predict(lowdim)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
plot = ax.scatter(xs=lowdim[:, 0],
            ys=lowdim[:, 1],
            zs=lowdim[:, 2],
            c=cluster_labels,
            marker=',', s=0.1)
plt.colorbar(plot)
plt.xlabel('Component 1')
plt.ylabel('Component 2')
print('Explained variance for each component: ')
for i, var in enumerate(mapper.explained_variance_ratio_):
    print('Component {}: {:.3f}'.format(i+1, var))

### Correlations

How strongly are different fields related to each other. Correlation ranges from -1 to +1. -1 means that as one value increases, the other decreases linearly. + 1 means both increase proportionally. A value close to zero means there is no linear relationship.

In [ ]:
correlations = np.corrcoef(df[power_cols], rowvar=False)
plt.matshow(correlations)
plt.colorbar()
plt.xticks(np.arange(len(power_cols)), power_cols, rotation=25)
plt.yticks(np.arange(len(power_cols)), power_cols, rotation=25);

## Questions

* What are the meanings of suffixes in the PowerDemand_* fields? do they correspond to floors, specific zones etc. in the building?
* How to handle missing cells in the data? Should those rows be ignored? Should they be interpolated?
* What other fields are needed to analyze power consumption of the building? Explore the BDX portal and surveys for the building to get an idea for the data available.